In [20]:
import os

os.getcwd()

'/methylation/F24A080000424_MUSekgzH_20240805100100'

In [21]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

root_dir="/methylation/F24A080000424_MUSekgzH_20240805100100/"  # 基因文件的根目录
sample_names=["13A","32A","50A","26A","28A","38A","74A","25A","75A","88A","97A"]     # 样本名称

# 报告存放的文件夹
if not os.path.exists(f"{root_dir}report/"):
    os.makedirs(f"{root_dir}report/")
for sample_name in sample_names:
    if not os.path.exists(f"{root_dir}report/{sample_name}"):
        os.makedirs(f"{root_dir}report/{sample_name}")
# 程序数据输出的文件夹
if not os.path.exists(f"{root_dir}output/"):
    os.makedirs(f"{root_dir}output/")

## 1. 数据基本处理与质控
将下机数据进行过滤，包括去污染，去测序接头和低质量碱基比例过高的reads，得到clean data。

In [22]:
# 表1 测序质量基本统计（公司提供）
report_table1=pd.read_csv(f'{root_dir}Clean_stat.xls', sep='\t', index_col=0)
report_table1

,Clean Reads,Clean Base,Read Length,Q20(%),Q30(%),GC(%)
Sample Name,,,,,,
13A,360085078,108025523400,PE150,95.58,90.74,22.42
25A,360265525,108079657500,PE150,96.62,89.73,22.86
26A,360191949,108057584700,PE150,96.84,90.23,22.36
28A,360244786,108073435800,PE150,95.83,91.18,22.61
32A,360173146,108051943800,PE150,96.72,92.74,22.68
38A,360332442,108099732600,PE150,97.92,95.14,21.63
50A,360257577,108077273100,PE150,97.89,95.11,21.61
74A,360194122,108058236600,PE150,95.39,88.52,21.66
75A,323495667,97048700100,PE150,95.07,87.86,22.01


In [23]:
# 图1：测序碱基含量分布
# 图2：碱基测序质量分布情况
# 公司已提供，位于每个样本文件夹中。

In [24]:
# 表2 比对结果统计

report_table2=[]
for sample_name in sample_names:
    data={}
    with open(f"{root_dir}{sample_name}/output/bismark_alignment/{sample_name}_1_bismark_bt2_PE_report.txt", "r") as file:
        for line in file:
            if line.count(":") == 1:   # 匹配包含1个冒号的行
                parts = line.split(":")
                try:
                    data[parts[0].strip()]=int(parts[1].strip())
                except:
                    pass
    item={}
    item["Sample Id"]=sample_name
    item["Clean Reads"]=data["Sequence pairs analysed in total"]
    item["Uniquely Mapped Reads"]=data["Number of paired-end alignments with a unique best hit"]
    item["Uniquely Mapping Rate"]=round(data["Number of paired-end alignments with a unique best hit"]/data["Sequence pairs analysed in total"]*100,2)

    # Bisulfite Conversion Rate的计算规则存疑
    # item["Bisulfite Conversion Rate"]=round((data["Total unmethylated C's in CpG context"]+data["Total unmethylated C's in CHG context"]+data["Total unmethylated C's in CHH context"])/data["Total number of C's analysed"]*100,2)
    item["Bisulfite Conversion Rate"]=round((data["Total unmethylated C's in CHG context"]+data["Total unmethylated C's in CHH context"])/(data["Total unmethylated C's in CHG context"]+data["Total unmethylated C's in CHH context"]+data["Total methylated C's in CHG context"]+data["Total methylated C's in CHH context"]+data["Total methylated C's in Unknown context"])*100,2)

    report_table2.append(item)
report_table2=pd.DataFrame(report_table2)
report_table2.to_csv(f"{root_dir}report/QC quality control table for each sample.tsv",sep="\t",index=False)
report_table2

,Sample Id,Clean Reads,Uniquely Mapped Reads,Uniquely Mapping Rate,Bisulfite Conversion Rate
0,13A,360085078,258302605,71.73,99.44
1,32A,360173146,279886483,77.71,99.43
2,50A,360257577,302777677,84.04,99.47
3,26A,360191949,272931733,75.77,99.44
4,28A,360244786,257555160,71.49,99.44
5,38A,360332442,297628490,82.60,99.46
6,74A,360194122,280008801,77.74,99.44
7,25A,360265525,278478393,77.30,99.42
8,75A,323495667,218869806,67.66,99.39
9,88A,354517702,261768036,73.84,99.40


In [25]:
# 图3 测序深度分布图

def plot_methylation_depth_distribution(sample_name):
    input_file=f"{root_dir}{sample_name}/output/{sample_name}_methylation_depth_report.txt"
    output_file=f"{root_dir}report/{sample_name}/Coverage of Corresponding Depth in Sample.jpg"
    df = pd.read_csv(input_file,sep="\t")
    df["C"]=df.sum(axis=1)
    df=df.set_index("Depth")
    df=df[["C","CG","CHG","CHH"]]
    df=df/df.sum(axis=0)
    df.plot()
    plt.title(f"Coverage of Corresponding Depth in Sample:{sample_name}")
    plt.ylabel("Fraction of Covered (%)")
    plt.legend(title='Context')
    plt.savefig(output_file,dpi=1000)
    plt.close()

for sample_name in sample_names:
    df_coverage=plot_methylation_depth_distribution(sample_name)

In [26]:
# 图4 C碱基测序深度的累积分布图
def plot_cumulative_methylation_depth_distribution(sample_name):
    input_file=f"{root_dir}{sample_name}/output/{sample_name}_methylation_depth_report.txt"
    output_file=f"{root_dir}report/{sample_name}/Cumulative Coverage of Corresponding Depth in Sample.jpg"
    df = pd.read_csv(input_file,sep="\t")
    df["C"]=df.sum(axis=1)
    df=df.set_index("Depth")
    df=df[["C","CG","CHG","CHH"]]
    df=df/df.sum(axis=0)
    df = df.cumsum() # 计算累积分布
    df.plot()
    plt.title(f"Cumulative Coverage of Corresponding Depth in Sample:{sample_name}")
    plt.ylabel("Fraction of Covered (%)")
    plt.legend(title='Context')
    plt.savefig(output_file,dpi=1000)
    plt.close()
    
for sample_name in sample_names:
    df_coverage=plot_cumulative_methylation_depth_distribution(sample_name)

In [27]:
# 表3、表4 样品在全基因组各染色体上的C位点覆盖度
def calc_coverage_rate_by_chromosome(sample_name):
    input_file=f"{root_dir}{sample_name}/output/{sample_name}_methylation_coverage_report.txt"
    output_file=f"{root_dir}report/{sample_name}/Coverage Rate Group By Chromosome.tsv"
    df = pd.read_csv(input_file,sep="\t")
    # 过滤只保留Chromosome列中以NC开头的行
    df = df[df['Chromosome'].str.startswith('NC')]
    # 计算每个 context 的覆盖率
    df['CoverageRate'] = round(df['covered'] / df['Count']*100,2)

    # 计算每个染色体的整体覆盖率
    overall_coverage = df.groupby('Chromosome').apply(lambda x: round(x['covered'].sum() / x['Count'].sum()*100,2),include_groups=False).reset_index()
    overall_coverage.columns = ['Chromosome', 'C (%)']

    # 创建以染色体为index、context为表头的透视表，并添加总体覆盖率列
    pivot_table = df.pivot(index='Chromosome', columns='Context', values='CoverageRate')

    # 将整体覆盖率加入透视表
    pivot_table = overall_coverage.merge(pivot_table, on='Chromosome')

    pivot_table.columns=["Chr","C (%)", "CG (%)", "CHG (%)", "CHH (%)"]

    # 保存结果到文件
    pivot_table.to_csv(output_file,sep="\t",index=False)
    return pivot_table
df_coverage_list=[]
for sample_name in sample_names:
    df_coverage=calc_coverage_rate_by_chromosome(sample_name)
    df_coverage.insert(0,"sample_name",sample_name)
    df_coverage_list.append(df_coverage)
df_coverage_list=pd.concat(df_coverage_list,axis=0).reset_index(drop=True)
df_coverage_list.to_csv(f"{root_dir}report/Coverage Rate Group By Chromosome.tsv",sep="\t",index=False)
df_coverage_list

,sample_name,Chr,C (%),CG (%),CHG (%),CHH (%)
0,13A,NC_000067.7,96.04,96.50,97.51,95.61
1,13A,NC_000068.8,95.44,96.28,96.91,94.98
2,13A,NC_000069.7,95.80,96.20,97.22,95.39
3,13A,NC_000070.7,94.51,95.22,95.94,94.06
4,13A,NC_000071.7,94.90,96.13,96.52,94.36
...,...,...,...,...,...,...
237,97A,NC_000084.7,96.35,97.05,97.84,95.90
238,97A,NC_000085.7,96.86,97.80,98.25,96.41
239,97A,NC_000086.8,88.95,87.82,89.95,88.74
240,97A,NC_000087.8,0.78,1.35,0.80,0.75


In [28]:
# 表5、表6 样品在全基因组各类型调控元件范围内的C位点覆盖度
# 暂时不做

In [29]:
# 表7 各样品QC质控表(去重之后的数据)
def get_report_table7(sample_name):
    import re
    PE_report={}
    with open(f"{root_dir}{sample_name}/output/bismark_alignment/{sample_name}_1_bismark_bt2_PE_report.txt", "r") as file:
        for line in file:
            if line.count(":") == 1:   # 匹配包含1个冒号的行
                parts = line.split(":")
                try:
                    PE_report[parts[0].strip()]=int(parts[1].strip())
                except:
                    pass
    
    # 查找deduplication_percentage
    percentage_pattern = re.compile(r"Total number duplicated alignments removed:.*?(\d+\.\d+)%")
    with open(f"{root_dir}{sample_name}/output/bismark_deduplicate/{sample_name}_1_bismark_bt2_pe.deduplication_report.txt", "r") as file:
        for line in file:
            match = percentage_pattern.search(line)
            if match:
                deduplication_percentage = match.group(1)
                break
    
    # 查找去重后的甲基化、非甲基化数据
    splitting_report={}
    with open(f"{root_dir}{sample_name}/output/bismark_methylation/{sample_name}_1_bismark_bt2_pe.deduplicated_splitting_report.txt", "r") as file:
        for line in file:
            if line.count(":") == 1:   # 匹配包含1个冒号的行
                parts = line.split(":")
                try:
                    splitting_report[parts[0].strip()]=int(parts[1].strip())
                except:
                    pass
    
    # 计算平均深度
    depth_report=pd.read_csv(f"{root_dir}{sample_name}/output/{sample_name}_methylation_depth_report.txt",sep="\t")
    depth_report["C"]=depth_report[["CG","CHG","CHH"]].sum(axis=1)
    depth_report["CxDepth"]=depth_report["C"]*depth_report["Depth"]
    avg_depth = (depth_report["CxDepth"].sum()/depth_report["C"].sum()).round(2)

    # 计算Coverage
    coverage_report=pd.read_csv(f"{root_dir}{sample_name}/output/{sample_name}_methylation_coverage_report.txt",sep="\t")
    coverage=(coverage_report["covered"].sum()/coverage_report["Count"].sum()*100).round(2)

    item={}
    item["Sample ID"]=sample_name
    item["Clean Data Size (bp)"]=PE_report["Sequence pairs analysed in total"]
    item["Mapping Rate (%)"]=round(PE_report["Number of paired-end alignments with a unique best hit"]/PE_report["Sequence pairs analysed in total"]*100,2)

    # Bisulfite Conversion Rate的计算规则存疑
    # item["Bisulfite Conversion Rate (%)"]=round((splitting_report["Total C to T conversions in CpG context"]+splitting_report["Total C to T conversions in CHG context"]+splitting_report["Total C to T conversions in CHH context"])/splitting_report["Total number of C's analysed"]*100,2)
    item["Bisulfite Conversion Rate"]=round((splitting_report["Total C to T conversions in CpG context"]+splitting_report["Total C to T conversions in CHH context"])/(splitting_report["Total C to T conversions in CHG context"]+splitting_report["Total C to T conversions in CHH context"]+splitting_report["Total methylated C's in CHG context"]+splitting_report["Total methylated C's in CHH context"])*100,2)
    item["Duplication Rate (%)"]=deduplication_percentage
    item["Average Depth (X)"]=avg_depth
    item["Coverage (%)"]=coverage
    return item

report_table7=[]
for sample_name in sample_names:
    item = get_report_table7(sample_name)
    report_table7.append(item)
report_table7=pd.DataFrame(report_table7)
report_table7.to_csv(f"{root_dir}report/QC quality control table for each sample (deduplicated).tsv",sep="\t",index=False)
report_table7


,Sample ID,Clean Data Size (bp),Mapping Rate (%),Bisulfite Conversion Rate,Duplication Rate (%),Average Depth (X),Coverage (%)
0,13A,360085078,71.73,77.43,10.06,10.81,92.07
1,32A,360173146,77.71,77.43,9.89,11.86,92.31
2,50A,360257577,84.04,78.40,5.15,13.12,92.75
3,26A,360191949,75.77,77.59,6.78,12.24,92.58
4,28A,360244786,71.49,77.27,8.85,10.95,91.71
5,38A,360332442,82.60,78.39,5.54,12.91,93.38
6,74A,360194122,77.74,78.19,5.06,12.13,92.14
7,25A,360265525,77.30,77.01,5.90,13.06,92.64
8,75A,323495667,67.66,77.50,4.95,9.72,91.89
9,88A,354517702,73.84,76.91,4.75,12.00,92.12


##  2. 全基因组甲基化水平分析
用于分析的DNA样品为多细胞样品，因此C碱基的甲基化水平是一个0% ～ 100%范围内的数值，等于该C碱基上覆盖到的支持mC的序列数除以有效覆盖的序列总数，通常CG甲基化存在于基因和重复序列中，在基因表达调控过程中起到非常重要的作用。非CG类型的序列（CHG和CHH）在基因中十分少见，主要存在于基因间区和富含重复序列的区域，在沉默转座子过程中起关键作用。

In [30]:
# 表8、表9 样品全基因组及各染色体的平均甲基化水平

def calc_methylation_level_by_chromosome(sample_name):
    input_file=f"{root_dir}{sample_name}/output/{sample_name}_methylation_coverage_report.txt"
    output_file=f"{root_dir}report/{sample_name}/Methylation Level Groupp By Chromosome.tsv"

    # 读取表格文件
    df = pd.read_csv(input_file, sep="\t")

    # 过滤只保留Chromosome列中以NC开头的行
    df = df[df['Chromosome'].str.startswith('NC')]

    # 计算每个 context 的甲基化水平
    df['MethylationLevel'] = round(df['totalReadsM'] / df['totalReadsN'] * 100, 2)

    # 创建以染色体为index、context为表头的透视表
    pivot_table = df.pivot(index='Chromosome', columns='Context', values='MethylationLevel')

    # 计算每个染色体的总体甲基化水平
    overall_methylation = df.groupby('Chromosome').apply(
        lambda x: round(x['totalReadsM'].sum() / x['totalReadsN'].sum() * 100, 2),
        include_groups=False
    ).reset_index()
    overall_methylation.columns = ['Chromosome', 'C (%)']

    # 将总体甲基化水平加入透视表
    pivot_table = overall_methylation.merge(pivot_table, on='Chromosome')

    pivot_table=pivot_table.fillna(0)

    pivot_table.columns=["Chr","C (%)", "CG (%)", "CHG (%)", "CHH (%)"]

    # 保存结果到文件
    pivot_table.to_csv(output_file,sep="\t",index=False)

    return pivot_table

df_methylation_level_list=[]
for sample_name in sample_names:
    df_methylation_level=calc_methylation_level_by_chromosome(sample_name)
    # df_methylation_level["sample_name"]=sample_name
    df_methylation_level.insert(0,"sample_name",sample_name)
    df_methylation_level_list.append(df_methylation_level)
df_methylation_level_list=pd.concat(df_methylation_level_list,axis=0).reset_index(drop=True)
df_methylation_level_list.to_csv(f"{root_dir}report/Methylation Level Groupp By Chromosome.tsv",sep="\t",index=False)
df_methylation_level_list

,sample_name,Chr,C (%),CG (%),CHG (%),CHH (%)
0,13A,NC_000067.7,3.50,75.93,0.54,0.52
1,13A,NC_000068.8,3.84,75.19,0.55,0.54
2,13A,NC_000069.7,3.53,75.84,0.54,0.52
3,13A,NC_000070.7,3.74,74.62,0.55,0.53
4,13A,NC_000071.7,3.92,75.08,0.55,0.53
...,...,...,...,...,...,...
237,97A,NC_000084.7,4.09,79.04,0.59,0.59
238,97A,NC_000085.7,4.35,77.00,0.63,0.64
239,97A,NC_000086.8,3.12,77.14,0.54,0.52
240,97A,NC_000087.8,9.46,83.24,0.64,0.57


In [31]:
# 表10 表11 样品在全基因组各类型调控元件范围内的甲基化水平
# 暂时不做

In [33]:
import seaborn as sns
from matplotlib.ticker import ScalarFormatter,FuncFormatter

In [34]:
# 绘制M-bias图
def plot_mbias(mbias_file,output_dir):
    # 读取整个文件内容
    with open(mbias_file, 'r') as file:
        content = file.read()

    # 定义正则表达式查找不同上下文表格
    # 正则表达式将匹配上下文标题、分隔符、表头和表数据
    pattern = r'(\w+) context \((\w+)\)\n=+\n([\s\S]+?)(?=\n[A-Z]|$)'
    matches = re.findall(pattern, content)

    df = []
    # 解析匹配到的表格
    for context, read, table_data in matches:
        # 将表格转换为 DataFrame
        item = pd.read_csv(StringIO(table_data), sep='\t')
        # 添加上下文和read标签列到 DataFrame
        item.insert(0,'context',context)
        item.insert(1,'read',read)
        df.append(item)

    df=pd.concat(df)
    for read_name in df['read'].unique():
        # 创建一个图表
        fig, ax1 = plt.subplots(figsize=(8, 4))

        # 创建组合字段
        df['read_context'] = df['read'] + ' - ' + df['context']

        # 绘制甲基化率（左侧 Y 轴）
        sns.lineplot(data=df[df["read"]==read_name], x='position', y='% methylation', hue='context', linewidth=3, ax=ax1)
        
        ax1.grid(True, linestyle='--', linewidth=0.7, color='#dddddd')
        ax1.set_xlabel("Position in Read [bp]")
        # 设置 x 和 y 轴从 0 开始
        ax1.set_xlim(left=0,right=df["position"].max())
        ax1.set_ylim(bottom=0,top=100)
        # ax1.set_ylabel("% Methylation", color='tab:blue')
        # ax1.tick_params(axis='y', labelcolor='tab:blue')
        ax1.set_ylabel("% Methylation")

        # 创建第二个 Y 轴（右侧）
        ax2 = ax1.twinx()  # 共享 x 轴

        # 绘制甲基化数量（右侧 Y 轴）
        sns.lineplot(data=df[df["read"]==read_name], x='position', y='count methylated', hue='context', linewidth=1, linestyle='--', ax=ax2)

        # 设置第二个 Y 轴
        # ax2.set_ylabel("# Methylation Calls", color='tab:orange')
        # ax2.tick_params(axis='y', labelcolor='tab:orange')
        ax2.set_ylabel("# Methylation Calls")
        ax2.set_ylim(bottom=0)

        # 自定义格式化函数
        def format_func(value, tick_number):
            if value >= 10_000_000:
                return f'{value / 1_000_000:.0f}M'
            elif value >= 1_000_000:
                return f'{value / 1_000_000:.1f}M'
            elif value >= 10_000:
                return f'{value / 1_000:.0f}K'
            elif value >= 1_000:
                return f'{value / 1_000:.1f}K'
            else:
                return f'{value:.0f}'
        ax2.yaxis.set_major_formatter(FuncFormatter(format_func))

        read_fulname={"R1":"Read 1","R2": "Read 2"}[read_name]
        plt.title(f"{sample_name} {read_fulname}")

        # 显示图例
        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        labels = [label + " methylation" for label in labels]
        labels2 = [label + " total calls" for label in labels2]
        ax1.legend(lines + lines2, labels + labels2,loc="upper right")
        ax2.get_legend().remove()

        plt.savefig(f"{output_dir}M-bias {read_name}.jpg",dpi=1000)
        # plt.show()
        plt.close()
for sample_name in sample_names:
    mbias_file = f"{root_dir}{sample_name}/output/bismark_methylation/{sample_name}_1_bismark_bt2_pe.deduplicated.M-bias.txt"
    output_dir=f"{root_dir}report/{sample_name}/"
    plot_mbias(mbias_file, output_dir)

## 3. 甲基化 C碱基中 CG, CHG 与CHH的分布比例
mCG，mCHG和mCHH三种碱基类型的构成比例在不同物种中，甚至在同一物种不同样品中都存在很大差异。因此，不同时间、空间、生理条件下的样品会表现出不同的甲基化图谱，各类型mC( mCG、mCHG和mCHH )的数目，及其在全部mC的位点中所占的比例，在一定程度上反映了特定物种的全基因组甲基化图谱的特征。mCG、mCHG和mCHH分别表示表示甲基化CG、甲基化CHG和甲基化CHH。三种碱基类型占比总和为100%，甲基化C鉴定方法依据Lister的文章描述进行。

In [35]:
# 表12 mCG、mCHG和mCHH三种类型甲基化胞嘧啶的比例

report_table12=[]
for sample_name in sample_names:
    data={}
    with open(f"{root_dir}{sample_name}/output/bismark_methylation/{sample_name}_1_bismark_bt2_pe.deduplicated_splitting_report.txt", "r") as file:
        for line in file:
            if line.count(":") == 1:   # 匹配包含1个冒号的行
                parts = line.split(":")
                try:
                    data[parts[0].strip()]=int(parts[1].strip())
                except:
                    pass
    item={}
    item["Sample Id"]=sample_name
    item["mCG"]=data["Total methylated C's in CpG context"]
    item["mCHG"]=data["Total methylated C's in CHG context"]
    item["mCHH"]=data["Total methylated C's in CHH context"]
    total_mC=item["mCG"]+item["mCHG"]+item["mCHH"]
    item["mCG proportion (%)"]=round(item["mCG"]/total_mC*100,2)
    item["mCHG proportion (%)"]=round(item["mCHG"]/total_mC*100,2)
    item["mCHH proportion (%)"]=round(item["mCHH"]/total_mC*100,2)

    
    report_table12.append(item)
report_table12=pd.DataFrame(report_table12).set_index("Sample Id")
report_table12.to_csv(f"{root_dir}report/Proportions of Three Types of Methylated Cytosine.tsv",sep="\t")

report_table12

,mCG,mCHG,mCHH,mCG proportion (%),mCHG proportion (%),mCHH proportion (%)
Sample Id,,,,,,
13A,353582218,13466905,43179108,86.19,3.28,10.53
32A,380305619,14970374,48448057,85.71,3.37,10.92
50A,385841393,15132609,51331234,85.31,3.35,11.35
26A,409455527,15141590,49161851,86.43,3.20,10.38
28A,369382134,13577988,43129274,86.69,3.19,10.12
38A,395482471,15185144,51921364,85.49,3.28,11.22
74A,386013982,14606916,49230271,85.81,3.25,10.94
25A,463998830,17218295,55282291,86.49,3.21,10.30
75A,330761705,12936961,42390800,85.67,3.35,10.98


In [36]:
# 图5 不同序列类型甲基化C碱基的分布比例
for sample_name,item in report_table12[["mCG","mCHG","mCHH"]].iterrows():
    item.plot(kind='pie', autopct='%1.1f%%', figsize=(5,5))
    plt.title(f'The proportion of every type mC (Sample: {sample_name})')
    plt.legend()
    plt.ylabel("")
    plt.tight_layout()
    plt.savefig(f"{root_dir}report/{sample_name}/The proportion of every type mC (Sample: {sample_name}).jpg",dpi=1000)
    plt.close()

## 4. 甲基化 CG、CHG和CHH的甲基化水平分布
不同类型的C碱基(mCG、mCHG和mCHH )，其甲基化水平在不同物种间，甚至同一物种不同细胞类型不同条件下其甲基化水平都存在差异。此图统计每种类型( CG、CHG和CHH )甲基化C的甲基化水平分布，反映了该物种DNA甲基化特征

In [138]:
# 绘制甲基化水平分布图（按每10%分组）
def bin_methylation_levels(df, range_size=10):
    """将数据分组到区间"""
    # 创建 bin 的区间
    bin_edges = np.arange(0, 101, range_size)
    bin_labels = [f"{i+range_size}" for i in bin_edges[:-1]]
    df['methylation_level_range'] = pd.cut(df['methylation_level'], bins=bin_edges, labels=bin_labels, right=False)

    # 计算每个 methylation_level_range 中的 count 总和
    range_df = df.groupby(['context', 'methylation_level_range'],observed=False)['count'].sum().reset_index()
    return range_df

def plot_methylation_level_distribution(sample_name):
    input_file=f"{root_dir}{sample_name}/output/{sample_name}_methylation_distribution_report.txt"
    # 读取TSV格式的输出文件
    df = pd.read_csv(input_file, sep='\t')
    df = df[df["methylation_level"]>0]

    # 确保数据按 context 和 methylation_level 排序
    df.sort_values(by=['context', 'methylation_level'], inplace=True)

    # 将数据分组到指定的区间
    df = bin_methylation_levels(df, range_size=10)

    # 从长格式转为宽格式
    df = df.pivot(index='methylation_level_range', columns='context', values='count')
    df.columns.name=None
    df.index.name=None

    # 计算总甲基化数量
    df["C"]=df[["CG","CHG","CHH"]].sum(axis=1)
    df=df[["C","CG","CHG","CHH"]]
    df = df/df.sum()*100  # 计算不同甲基化水平的占比
    
    # 创建绘图
    df.plot(marker='o',figsize=(6, 4))
    plt.xlabel('Methylation Level')
    plt.ylabel('Percentage (%)')
    plt.title(f'Methylation Level Distribution (Sample: {sample_name})')
    plt.legend(title='Context')
    plt.grid(True, linewidth=1, color='#f6f6f6')
    plt.tight_layout()
    plt.savefig(f"{root_dir}report/{sample_name}/Methylation Level Distribution (Sample: {sample_name}).jpg",dpi=1000)
    plt.close()

for sample_name in sample_names:
    plot_methylation_level_distribution(sample_name)

In [137]:
# 绘制甲基化水平累积分布图（按每1%分组）
def plot_cumulative_methylation_level_distribution(sample_name):
    input_file=f"{root_dir}{sample_name}/output/{sample_name}_methylation_distribution_report.txt"
    # 读取TSV格式的输出文件
    df = pd.read_csv(input_file, sep='\t')
    df = df[df["methylation_level"]>0]

    # 确保数据按 context 和 methylation_level 排序
    df.sort_values(by=['context', 'methylation_level'], inplace=True)

    # 从长格式转为宽格式
    df = df.pivot(index='methylation_level', columns='context', values='count')
    df.columns.name=None
    df.index.name=None

    # 计算总甲基化数量
    df["C"]=df[["CG","CHG","CHH"]].sum(axis=1)
    df=df[["C","CG","CHG","CHH"]]
    df = df/df.sum()*100  # 计算不同甲基化水平的占比
    df = df.cumsum() # 计算累积分布
    
    # 创建绘图
    df.plot(figsize=(6, 4))
    plt.xlabel('Methylation Level')
    plt.ylabel('Percentage (%)')
    plt.title(f'Cumulative Methylation Level Distribution (Sample: {sample_name})')
    plt.legend(title='Context')
    plt.grid(True, linewidth=1, color='#f6f6f6')
    plt.tight_layout()
    plt.savefig(f"{root_dir}report/{sample_name}/Cumulative Methylation Level Distribution (Sample: {sample_name}).jpg",dpi=1000)
    plt.close()


for sample_name in sample_names:
    plot_cumulative_methylation_level_distribution(sample_name)

## 5. 甲基化的CG，CHG，CHH附近碱基的序列特征分析
在一些真核生物中，甲基化位点附近碱基的序列特征，对反映甲基化发生的序列偏向有指导意义。为了研究序列特征与甲基化偏向性之间的联系，我们计算了甲基化位点上下游9个碱基 （mC位于第四个碱基）的甲基化百分比。

## 6. 染色体水平的甲基化C碱基密度分布
有研究证明非CG型的甲基化与CG型甲基化C的密度有很大的差异。染色体亚端粒区域DNA甲基化水平通常较高，这一现象与端粒长度以及重组有十分重要的作用，此外还有基因表达和蛋白与DNA的相互作用都有紧密联系。 如下图的显示，整体的甲基化水平图谱显示DNA甲基化的密度在整条染色体上变化很大。

## 7. 基因组的不同区域的甲基化分布特征

## 8. 基因组不同转录元件中的DNA平均甲基化水平
（暂时不绘制）

## 9. DMR的检测
差异甲基化区域（DMRs）是指不同样品中基因组表现出不同的甲基化模式的某些DNA片段。 DMR与遗传印记相关，在个体中表现为与父本或母本的甲基化状态一致。甲基化的等位基因经常表现为沉默状态。亲本与子代甲基化模式的差异常常导致表观遗传缺陷，而人工繁殖技术可能会导致异常甲基化的比例升高，并导致疾病的发生。

我们用滑动窗口的方法检测DMR。在两个样品基因组相同位置上寻找包含至少5个CG（或CHG，或CHH）的窗口，比较该窗口在两个样品数据中甲基化水平的差异，寻找在两个样品中甲基化水平有差异的区域。

In [38]:
# DMR数据统计（R代码中也实现了这个功能）
# def getDMRsStatistics(file_input,file_output):
#     columns=[
#         "seqnames", "start", "end", "width", "strand", "sumReadsM1", "sumReadsN1",
#         "proportion1", "sumReadsM2", "sumReadsN2", "proportion2", "cytosinesCount",
#         "context", "direction", "pValue", "regionType"
#     ]
#     df_DMRs=pd.read_csv(file_input,sep="\t",header=None,names=columns)
#     df_DMRs_Statistics=df_DMRs.groupby(["context","seqnames"]).size().reset_index(name='DMR number')
#     df_DMRs_Statistics.to_csv(file_output,sep="\t",index=False)
#     return df_DMRs_Statistics

# file_input = f"{root_dir}output/DMRsReplicatesBins.chr.txt"
# file_output = f"{root_dir}report/DMRsummary.tsv"
# df_DMRs_grouped = getDMRsStatistics(file_input,file_output)

In [39]:
# # 读取DMR分析结果，并输出对应的基因列表（R代码中也实现了这个功能）
# def extract_gene_info(file_path):
#     import re

#     # 初始化一个空的列表来存储提取的信息
#     dmr_genes = []
    
#     # 打开文件并逐行读取
#     with open(file_path, 'r') as file:
#         for line in file:
#             # 使用正则表达式分别匹配gene_id, gene_type, gene_name
#             gene_id_match = re.search(r'gene_id "([^"]+)";', line)
#             gene_type_match = re.search(r'gene_type "([^"]+)";', line)
#             gene_name_match = re.search(r'gene_name "([^"]+)";', line)
            
#             # 创建一个字典来存储当前行的匹配结果
#             gene_info = {}
#             if gene_type_match:
#                 gene_info['gene_type'] = gene_type_match.group(1)
#             if gene_name_match:
#                 gene_info['gene_name'] = gene_name_match.group(1)
#             if gene_id_match:
#                 gene_info['gene_id'] = gene_id_match.group(1)
            
#             # 如果三者都匹配到，才加入列表
#             if len(gene_info) == 3:
#                 dmr_genes.append(gene_info)


#     # 将提取的信息转换为DataFrame，并去除重复项，按照gene_type, gene_name, gene_id排序
#     dmr_genes = pd.DataFrame(dmr_genes).drop_duplicates().sort_values(by=['gene_type', 'gene_name', 'gene_id']).reset_index(drop=True)

#     # 使用 str.split 分割 gene_id 列
#     dmr_genes[['gene_id', 'version']] = dmr_genes['gene_id'].str.split('.', 1, expand=True)
    
#     return dmr_genes

# file_path = root_dir+'output/DMR_gene_association.bed'
# dmr_genes = extract_gene_info(file_path)
# dmr_genes.to_csv(root_dir+'report/DMR_genes.tsv',sep="\t",index=False)
# dmr_genes